In [ ]:
# import numpy as np
# from functools import reduce
import warnings

import pandas as pd

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 80)
pd.set_option('display.max_rows', 55)

warnings.simplefilter(action='ignore', category=FutureWarning)

open_stock = '../data/inzi/stock.xlsb'

files = [
    '../data/inzi/02 SEP/01-09-2021_IN-OUT DAILY REPORT.xlsb',
    '../data/inzi/03 SEP/02-09-2021_IN-OUT DAILY REPORT.xlsb',
    '../data/inzi/04 SEP/03-09-2021_IN-OUT DAILY REPORT.xlsb',
]

out_put_file = '../output/concet.xlsx'


stock = pd.read_excel(open_stock, header=2)

stock.rename(columns={
    'Unnamed: 5': 'Materials'
}, inplace=True)
stock['OPENING STOCK'] = stock['Unrestricted'] + \
    stock['Inspection'] + stock['Returned'] + stock['Blocked']
stock = stock[['Materials', 'OPENING STOCK']]
stock.rename(columns={'Materials': 'Material'}, inplace=True)
stock = stock.groupby(stock['Material']).aggregate(
    'sum')


def get_concat(sheet):
    dfs = []
    for file_name in files:
        df = pd.read_excel(file_name, sheet_name=sheet, header=8)
        df.drop(columns=['Unnamed: 0', 'Unnamed: 2',
                         'IN', 'OUT'], inplace=True)
        df.rename(columns={'Unnamed: 1': 'Material'}, inplace=True)
        if 'Unnamed: 5' in df:
            df.rename(columns={'Unnamed: 5': 'PURCHASE'}, inplace=True)
        dfs.append(df)

    final_result = pd.concat(dfs)
    final_result['temp'] = 0
    final_result = final_result[['Material', 'temp']]
    final_result.drop_duplicates(inplace=True)
    final_result = pd.merge(final_result, stock, how="left", on=['Material'], sort=False,
                            indicator=False, validate=None)
    del final_result['temp']
    l = len(dfs)

    for _ in range(l):
        in_stock = [344, 401, 623, 720, 321, 323, 327, 602]
        sum_col = []
        out_stock = [401, 720]
        subtract_col = []

        dfs[0] = pd.merge(dfs[0], final_result[final_result.columns[[0, -1]]], how="outer", on=['Material'], sort=False,
                          indicator=False, validate=None)
        dfs[0].set_axis([*dfs[0].columns[:-1], 'temporary'],
                        axis=1, inplace=True)

        for column in in_stock:
            if column in dfs[0].columns:
                sum_col.append(column)
        sum_col.append('temporary')

        for column in out_stock:
            if column in dfs[0].columns:
                subtract_col.append(column)

        dfs[0]['CLOSING STOCK'] = dfs[0][sum_col].sum(
            axis=1) - dfs[0][subtract_col].sum(axis=1)
        del dfs[0]['temporary']

        final_result = pd.merge(final_result, dfs[0], how="outer", on=['Material'], sort=False,
                                indicator=False, validate=None)
        del dfs[0]
    print(f'Concat {l} files together in {sheet}')
    # final_result.drop(final_result.columns[-1], axis=1, inplace=True)
    final_result.fillna(0, inplace=True)
    return final_result


raw_concat = get_concat('REPORT-RAW')
wip_concat = get_concat('REPORT-WIP')
fg_concat = get_concat('REPORT-FG')

In [ ]:
import warnings

import pandas as pd

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 80)
pd.set_option('display.max_rows', 55)



previous = pd.read_excel('../data/inzi/08 SEP/07-09-2021_IN-OUT-STOCK_DAILY REPORT.xlsb', sheet_name='REPORT_RAW', header=10)
today = pd.read_excel('../data/inzi/08 SEP/07-09-2021_IN-OUT DAILY REPORT.xlsb', sheet_name='REPORT-RAW', header=8)


result = pd.concat([previous, today])

result.rename(columns={
    'Unnamed: 1': 'Material'
}, inplace=True)

result['temp'] = 0

result = result[['Material', 'temp']]

result.drop_duplicates(inplace=True, keep='first')

# result = pd.merge(result, previous, how="left", on=['Material'], sort=False,
#                             indicator=False, validate=None)
# result = pd.merge(result, today, how="left", on=['Material'], sort=False,
#                             indicator=False, validate=None)
del result['temp']